In [1]:
# The Flowserv class provides the clinet that allows
# to install and run workflow templates.
from flowserv.client import Flowserv

# When creating an instance of the client the user can
# provide a base directory where all workflow files will be
# stored. If no directory is given a temporary directory
# will be created. If the 'clear' flag is True all existing
# files in the based directory will be removed.
# By setting the open access flag to 'True' we avoid having
# to authenticate a user before instealling and running a
# workflow.
client = Flowserv(basedir='.flowserv', open_access=True, clear=True)

In [2]:
from flowserv.client import FlowservRepo

# Get a list of workflow templates that are available from
# the flowserv repository.
for template_id, description, _ in FlowservRepo().list():
    print('{}\t{}'.format(template_id, description))

helloworld	Hello World Demo
toptagger	Top Tagger Demo - ML4Jets
piesingle	Analyze single colony image using PIE


In [3]:
# Install the 'Hello World!' template in the created environment.
# The template is installed directly from the flowserv workflow
# repository. For this example we ignore the post-processing step.
# Note that we explicitly specify the workflow identifier. This
# implies that running this notebook cell again (without first
# recreating the client) will raise an exception for duplicate
# workflow identifiers. To avoid this problem omit the identifier
# argument and let flowserv create a new unique identifier for the
# installed workflow.
workflow_id = client.install('helloworld', identifier='helloworld', ignore_postproc=True, verbose=True)

print('installed workflow with id {}'.format(workflow_id))

cloning into /tmp/tmp73rlsfe6
Copy files from local file system at /tmp/tmp73rlsfe6 to local file system at /home/heiko/.cache/flowserv
copied ngrams.py to helloworld/static/code/ngrams.py
copied analyze.py to helloworld/static/code/analyze.py
copied postproc.py to helloworld/static/code/postproc.py
copied helloworld.py to helloworld/static/code/helloworld.py
installed workflow with id helloworld


In [4]:
# Get the workflow handle.

workflow = client.open(workflow_id)

print(workflow.name())
print()
print(workflow.description())
print()
print(workflow.instructions())

Hello World Demo

Hello World Demo for ROB

This example benchmark is adopted from the REANA Hello World Demo. The workflow has a single step that takes a text file with person names as input, together with a greeting phrase, and a sleep time. For each name in the input file a greeting will be written to an output file that is the concatenation of the greeting phrase and the name. For the purpose of this demo the result file is then analyzed to compute a score for every workflow run. Analysis computes the number of distinct 3-grams in the output file. The overall score is the number of 3-grams divided by the number of lines in the output file. The goal is to achieve a high score.


In [5]:
# Run the 'Hello World' workflow. The list of input names
# can be read from a file or given as an io buffer.
from io import StringIO

names = StringIO('\n'.join(['Alice', 'Bob', 'Claire', 'Dave']))
args = dict({
    'greeting': 'Hey there',
    'sleeptime': 2,
    'names': names
})

run = workflow.start_run(args)

print('Workflow finished with {}'.format(run))

run with {'inputfile': 'data/names.txt', 'outputfile': 'results/greetings.txt', 'sleeptime': 2.0, 'greeting': 'Hey there'}
Workflow finished with SUCCESS


In [6]:
# List names of all run result files.

for f in run.files():
    print(f.name)

results/greetings.txt
results/analytics.json


In [7]:
# Print greetings file.

with run.get_file('results/greetings.txt').load().open() as f:
    for line in f:
        print(line.strip())

b'Hey there Alice'
b'Hey there Bob'
b'Hey there Claire'
b'Hey there Dave'


In [8]:
# Erase will delete the base directory.
client.erase()